In [1]:
!pip install opencv-python

In [2]:
pip install mediapipe-model-maker==0.2.1.3

Defaulting to user installation because normal site-packages is not writeable
  Using cached mediapipe_model_maker-0.2.1.3-py3-none-any.whl (127 kB)
  Using cached tensorflow_hub-0.16.1-py2.py3-none-any.whl (30 kB)
  Using cached absl_py-2.1.0-py3-none-any.whl (133 kB)
  Using cached tf_models_official-2.16.0-py2.py3-none-any.whl (2.7 MB)
  Using cached tensorflow_datasets-4.9.3-py3-none-any.whl (5.0 MB)
  Using cached tensorflow_addons-0.23.0-cp39-cp39-macosx_11_0_arm64.whl (12.6 MB)
  Using cached mediapipe-0.10.11-cp39-cp39-macosx_11_0_universal2.whl (50.4 MB)
  Using cached tensorflow-2.16.1-cp39-cp39-macosx_12_0_arm64.whl (227.0 MB)
  Using cached attrs-23.2.0-py3-none-any.whl (60 kB)
  Using cached jax-0.4.26-py3-none-any.whl (1.9 MB)
  Using cached jaxlib-0.4.26-cp39-cp39-macosx_11_0_arm64.whl (66.7 MB)
  Using cached matplotlib-3.8.4-cp39-cp39-macosx_11_0_arm64.whl (7.5 MB)
  Using cached sounddevice-0.4.6-py3-none-macosx_10_6_x86_64.macosx_10_6_universal2.whl (107 kB)
  Using 

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [ ]:
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 

In [ ]:
cap = cv2.VideoCapture(0)

# Curl counter variables
Lcounter = 0 
Rcounter = 0
Lstage = None
Rstage = None

## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates
            Lhip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            Lknee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            Lankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
            # Get coordinates
            Rhip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            Rknee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
            Rankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
            
            # Calculate angle
            Langle = calculate_angle(Lhip, Lknee, Lankle)
             # Calculate angle
            Rangle = calculate_angle(Rhip, Rknee, Rankle)
            
            
            # Visualize angle
            cv2.putText(image, str(Langle), 
                           tuple(np.multiply(Lknee, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
            # Visualize angle
            cv2.putText(image, str(Rangle), 
                           tuple(np.multiply(Rankle, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                       )
            
              

            # Curl counter logic
            if Langle < 160 and Rangle < 160:
                Lstage = "down"
            elif Langle > 30 and Rangle > 30 and Lstage =='down':
                Lstage="up"
                Lcounter +=1
                print(Lcounter)
                
         
                       
        except:
            pass
        
        # Render curl counter
        # Setup status box
        cv2.rectangle(image, (0,0), (225,73), (245,117,16), -1)
        
        # Rep data
        cv2.putText(image, 'REPS', (15,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(Lcounter), 
                    (10,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        cv2.rectangle(image, (2000,0), (1600,73), (245,117,16), -1)
        cv2.putText(image, 'REPS', (15,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(Rcounter), 
                    (1800,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
         
        
        # Render detections
       # Render detections
        if (Lstage == "up"):
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(0,0,0), thickness=3, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(230,0,0), thickness=2, circle_radius=2) 
                                )  
        if (Lstage == "down"):
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(0,0,0), thickness=3, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(230,110,110), thickness=2, circle_radius=2) 
                                )  
        
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
    for i in range (1,5):
        cv2.waitKey(1)